# A. Polar dataframe

In [ ]:
!pip install polars
!pip install xlsx2csv
#polars-0.15.8
#xlsx2csv-0.8.0

In [ ]:
import polars as pl
excel_file = "./existencias-colmenas.xlsx"
a = pl.read_excel(
      file=excel_file, 
      sheet_id=0, 
      xlsx2csv_options = {
          "linedelimitator":"\n"
      }, 
    )

print(a)

{'13': shape: (28, 7)
┌──────┬───────────────┬───────────────┬──────────────┬──────────────┬──────────────┬──────────────┐
│      ┆ 13.  Número   ┆ _duplicated_0 ┆ _duplicated_ ┆ _duplicated_ ┆ _duplicated_ ┆ _duplicated_ │
│ ---  ┆ de Unidades   ┆ ---           ┆ 1            ┆ 2            ┆ 3            ┆ 4            │
│ str  ┆ Producti...   ┆ str           ┆ ---          ┆ ---          ┆ ---          ┆ ---          │
│      ┆ ---           ┆               ┆ str          ┆ str          ┆ str          ┆ str          │
│      ┆ str           ┆               ┆              ┆              ┆              ┆              │
╞══════╪═══════════════╪═══════════════╪══════════════╪══════════════╪══════════════╪══════════════╡
│ null ┆ null          ┆ null          ┆ null         ┆ Tipo de      ┆ null         ┆ null         │
│      ┆               ┆               ┆              ┆ Colmenas     ┆              ┆              │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌

## References

> [xlsx2csv](https://github.com/dilshod/xlsx2csv)

# C. Pandas dataframe

In [ ]:
# import polars as pl
import pandas as pd

excel_file = "./existencias-colmenas.xlsx"

pandas_excel = pd.read_excel(excel_file,sheet_name=0, skiprows=2, )
print(pandas_excel.head())

   Unnamed: 0                    Unnamed: 1        Unnamed: 2  \
0         NaN                           NaN               NaN   
1         NaN       Total Nacional - Región             Total   
2         NaN                           NaN  Número de UPA3,4   
3         NaN                Total Nacional              5909   
4         NaN  Región de Arica y Parinacota                 6   

           Unnamed: 3           Unnamed: 4          Unnamed: 5  \
0                 NaN     Tipo de Colmenas                 NaN   
1                 NaN  Colmenas Modernas 5                 NaN   
2  Número de colmenas       Número de UPA4  Número de colmenas   
3              333037                 4997              306758   
4                  53                    5                  49   

            Unnamed: 6          Unnamed: 7  
0                  NaN                 NaN  
1  Colmenas Rústicas 6                 NaN  
2       Número de UPA4  Número de colmenas  
3                  995          

# B. PySpark dataframe

In [34]:
!pip install pyspark
!pip install findspark
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Checking informatino session

In [35]:
import pandas as pd
import pyspark

import findspark
findspark.init()

In [36]:
# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
print(spark)

In [37]:
from pyspark import SparkContext
from pyspark.sql import SQLContext 

s_context = SparkContext.getOrCreate()
#s_context = SparkContext("local", "existencia_colmenas")
sql = SQLContext(s_context)

In [38]:
df = pd.read_excel("./existencias-colmenas.xlsx", sheet_name=0)
# Standardising all data types from pandas to String for spark
df = df.astype(str)

df_spark = sql.createDataFrame(df)


In [39]:
df_spark.show()

+----------+--------------------+----------------+------------------+-------------------+------------------+-------------------+------------------+
|Unnamed: 0|          Unnamed: 1|      Unnamed: 2|        Unnamed: 3|         Unnamed: 4|        Unnamed: 5|         Unnamed: 6|        Unnamed: 7|
+----------+--------------------+----------------+------------------+-------------------+------------------+-------------------+------------------+
|       nan|13.  Número de Un...|             nan|               nan|                nan|               nan|                nan|               nan|
|       nan|                 nan|             nan|               nan|                nan|               nan|                nan|               nan|
|       nan|                 nan|             nan|               nan|   Tipo de Colmenas|               nan|                nan|               nan|
|       nan|Total Nacional - ...|           Total|               nan|Colmenas Modernas 5|               nan|Colm

In [40]:
df_spark.printSchema()

root
 |-- Unnamed: 0: string (nullable = true)
 |-- Unnamed: 1: string (nullable = true)
 |-- Unnamed: 2: string (nullable = true)
 |-- Unnamed: 3: string (nullable = true)
 |-- Unnamed: 4: string (nullable = true)
 |-- Unnamed: 5: string (nullable = true)
 |-- Unnamed: 6: string (nullable = true)
 |-- Unnamed: 7: string (nullable = true)



In [41]:
df_spark.count()

31

In [42]:
df_spark.select("Unnamed: 0", "Unnamed: 1", "Unnamed: 2").show(4)

+----------+--------------------+----------+
|Unnamed: 0|          Unnamed: 1|Unnamed: 2|
+----------+--------------------+----------+
|       nan|13.  Número de Un...|       nan|
|       nan|                 nan|       nan|
|       nan|                 nan|       nan|
|       nan|Total Nacional - ...|     Total|
+----------+--------------------+----------+
only showing top 4 rows



In [43]:
# Adding SQL functions to count empty rows
from pyspark.sql import functions as F

# Counting nan Rows
df_spark.select(
    [
        F.count
          (
              F.when(
                  F.isnan(c), c
              )
          ).alias(c) 
        for c in df_spark.columns
     ]
  ).show()
F.when

+----------+----------+----------+----------+----------+----------+----------+----------+
|Unnamed: 0|Unnamed: 1|Unnamed: 2|Unnamed: 3|Unnamed: 4|Unnamed: 5|Unnamed: 6|Unnamed: 7|
+----------+----------+----------+----------+----------+----------+----------+----------+
|        31|         4|        12|        13|        11|        13|        12|        13|
+----------+----------+----------+----------+----------+----------+----------+----------+



<function pyspark.sql.functions.when(condition: pyspark.sql.column.Column, value: Any) -> pyspark.sql.column.Column>

In [44]:
# Replace nan with Null
df_spark = df_spark.replace('nan', None)
df_spark.show()

+----------+--------------------+----------------+------------------+-------------------+------------------+-------------------+------------------+
|Unnamed: 0|          Unnamed: 1|      Unnamed: 2|        Unnamed: 3|         Unnamed: 4|        Unnamed: 5|         Unnamed: 6|        Unnamed: 7|
+----------+--------------------+----------------+------------------+-------------------+------------------+-------------------+------------------+
|      null|13.  Número de Un...|            null|              null|               null|              null|               null|              null|
|      null|                null|            null|              null|               null|              null|               null|              null|
|      null|                null|            null|              null|   Tipo de Colmenas|              null|               null|              null|
|      null|Total Nacional - ...|           Total|              null|Colmenas Modernas 5|              null|Colm

In [45]:
# Drop rows with null values on selected columns

columns = df_spark.columns

new_df = df_spark.na.drop(
              subset=columns[2:],how="all"
            )
new_df.show(32)

+----------+--------------------+----------------+------------------+-------------------+------------------+-------------------+------------------+
|Unnamed: 0|          Unnamed: 1|      Unnamed: 2|        Unnamed: 3|         Unnamed: 4|        Unnamed: 5|         Unnamed: 6|        Unnamed: 7|
+----------+--------------------+----------------+------------------+-------------------+------------------+-------------------+------------------+
|      null|                null|            null|              null|   Tipo de Colmenas|              null|               null|              null|
|      null|Total Nacional - ...|           Total|              null|Colmenas Modernas 5|              null|Colmenas Rústicas 6|              null|
|      null|                null|Número de UPA3,4|Número de colmenas|     Número de UPA4|Número de colmenas|     Número de UPA4|Número de colmenas|
|      null|      Total Nacional|            5909|            333037|               4997|            306758|    

In [46]:
from pyspark.sql import functions as F
# Drop null columns
null_counts = new_df.select(
                [
                    F.count(
                        F.when(
                            F.col(a).isNull(), a
                            )
                        ).alias(a) 
                    for a in new_df.columns
                 ]
              ).collect()[0].asDict()

to_delete = [ p for p, u in null_counts.items() if u == new_df.count() ]

new_df = new_df.drop(*to_delete)
new_df.show(truncate=True)


+--------------------+----------------+------------------+-------------------+------------------+-------------------+------------------+
|          Unnamed: 1|      Unnamed: 2|        Unnamed: 3|         Unnamed: 4|        Unnamed: 5|         Unnamed: 6|        Unnamed: 7|
+--------------------+----------------+------------------+-------------------+------------------+-------------------+------------------+
|                null|            null|              null|   Tipo de Colmenas|              null|               null|              null|
|Total Nacional - ...|           Total|              null|Colmenas Modernas 5|              null|Colmenas Rústicas 6|              null|
|                null|Número de UPA3,4|Número de colmenas|     Número de UPA4|Número de colmenas|     Número de UPA4|Número de colmenas|
|      Total Nacional|            5909|            333037|               4997|            306758|                995|             26279|
|Región de Arica y...|               6|  

In [47]:
# Getting those columns with null field (in other words, headers)
import pyspark.sql.functions as f
from functools import reduce

cols = [F.col(c) for c in new_df.columns]
filter_expr = reduce(
    lambda a, b: a | b.isNull(), 
    cols[1:], 
    cols[0].isNull()
    )

header_df = new_df.filter(filter_expr)
header_df.show(truncate=False)

+-----------------------+----------------+------------------+-------------------+------------------+-------------------+------------------+
|Unnamed: 1             |Unnamed: 2      |Unnamed: 3        |Unnamed: 4         |Unnamed: 5        |Unnamed: 6         |Unnamed: 7        |
+-----------------------+----------------+------------------+-------------------+------------------+-------------------+------------------+
|null                   |null            |null              |Tipo de Colmenas   |null              |null               |null              |
|Total Nacional - Región|Total           |null              |Colmenas Modernas 5|null              |Colmenas Rústicas 6|null              |
|null                   |Número de UPA3,4|Número de colmenas|Número de UPA4     |Número de colmenas|Número de UPA4     |Número de colmenas|
+-----------------------+----------------+------------------+-------------------+------------------+-------------------+------------------+



In [109]:
# Header dataframe to header numpy
import numpy as np
import unidecode

def filling_data(multi_array): 
  for row in multi_array:
    last_value=None
    for i in range(len(row)):
      if ( row[i]==None and i>0 ):
        row[i] = last_value
      else:
        last_value = row[i]
  return multi_array

def data_normalization(xl_array):  
  """It will swap: uppercase to lowercase, 
  blanckspace to underscore, and other"""
  xl_array = np.char.lower(xl_array)
  xl_array = np.char.strip(xl_array)
  xl_array = np.char.replace(xl_array, "-", "")
  xl_array = np.char.replace(xl_array, "  ", " ")
  xl_array = np.char.replace(xl_array, " ", "_")
  xl_array = np.char.replace(xl_array, "none", "")
  xl_array = np.char.replace(xl_array, ",", "")
  ##self.xl_array = np.char.replace(self.xl_array, ".", "")
  ##self.xl_array = np.char.replace(self.xl_array, ",", "")
  return xl_array

# Removing non ascii and special characteres in columns
def ascii_ignore(x):
  return unidecode.unidecode(x)

# Mergin columns to unique ones
def mergin_rows(array):
  a = []
  for row in array:
    a.append(np.char.array(row))
  b =np.char.array(0)
  for x in a:
    b = b + "__" +x
  b = b.replace('0___', '')
  b = b.replace('0_', '')
  b = [ ascii_ignore(x) for x in b ]
  return b

header_np = np.array(header_df.select( header_df.columns ).collect())

header_np = mergin_rows(data_normalization(filling_data(header_np).astype(str)))

In [49]:
#removing null rows
data_df = new_df.na.drop(how="any")
data_df.show(truncate=False)

+---------------------------------------------------+----------+----------+----------+----------+----------+----------+
|Unnamed: 1                                         |Unnamed: 2|Unnamed: 3|Unnamed: 4|Unnamed: 5|Unnamed: 6|Unnamed: 7|
+---------------------------------------------------+----------+----------+----------+----------+----------+----------+
|Total Nacional                                     |5909      |333037    |4997      |306758    |995       |26279     |
|Región de Arica y Parinacota                       |6         |53        |5         |49        |2         |4         |
|Región de Tarapacá                                 |8         |48        |4         |18        |4         |30        |
|Región de Antofagasta                              |7         |25        |5         |21        |2         |4         |
|Región de Atacama                                  |30        |359       |24        |332       |6         |27        |
|Región de Coquimbo                     

In [111]:
data_df.toDF(*header_np).show(truncate=False)


+---------------------------------------------------+----------------------+-------------------------+-----------------------------------------------------+---------------------------------------------------------+-----------------------------------------------------+---------------------------------------------------------+
|total_nacional_region__                            |total__numero_de_upa34|total__numero_de_colmenas|tipo_de_colmenas__colmenas_modernas_5__numero_de_upa4|tipo_de_colmenas__colmenas_modernas_5__numero_de_colmenas|tipo_de_colmenas__colmenas_rusticas_6__numero_de_upa4|tipo_de_colmenas__colmenas_rusticas_6__numero_de_colmenas|
+---------------------------------------------------+----------------------+-------------------------+-----------------------------------------------------+---------------------------------------------------------+-----------------------------------------------------+---------------------------------------------------------+
|Total Nacional    

In [50]:
# Calculating Pi values using PySpark
import random
NUM_SAMPLES = 100000000
def inside(p):
  x, y = random.random(), random.random()
  return x*x + y*y < 1

count = s_context.parallelize(range(0, NUM_SAMPLES)).filter(inside).count()
pi = 4 * count / NUM_SAMPLES
print("Pi is roughly", pi)

Pi is roughly 3.14162312


# References




*   [Install PySpark 3 on Google Colab the Easy Way](https://medium.com/grabngoinfo/install-pyspark-3-on-google-colab-the-easy-way-577ec4a2bcd8)
*   [Introduction to PySpark](https://medium.com/the-researchers-guide/introduction-to-pyspark-a61f7217398e#:~:text=The%20first%20step%20would%20be,perform%20data%20loading%20and%20manipulations.&text=The%20next%20step%20is%20to,and%20instantiating%20with%20getOrCreate(%20).)
*   [How to Get Started with PySpark: compute power](https://towardsdatascience.com/how-to-get-started-with-pyspark-1adc142456ec?gi=10f32e5a2155)
*   [Dataframes and Big Data](https://medium.com/analytics-vidhya/introduction-to-pyspark-take-your-first-steps-into-big-data-analytics-b98cf7dce8b3)










## D. Cleaning messy data whyqd

In [ ]:
!pip install whyqd
#whyqd-0.6.2 
#xlrd-2.0.1

In [ ]:
import whyqd
SCHEMA_SOURCE = "/full/path_to/2020_rates_data_schema.json"
DIRECTORY = "/path_to/working/directory/"
    # Note: these links may no longer work when you follow this tutorial. Get the latest ones...
INPUT_DATA = [
            "https://www.portsmouth.gov.uk/ext/documents-external/biz-ndr-properties-january-2020.xls",
            "https://www.portsmouth.gov.uk/ext/documents-external/biz-ndr-reliefs-january-2020.xls",
            "https://www.portsmouth.gov.uk/ext/documents-external/biz-empty-commercial-properties-january-2020.xls"]
method = whyqd.Method(directory=DIRECTORY, schema=SCHEMA)
method.add_data(source=INPUT_DATA)